# 18. Training Models w/o TL weights
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mcqueg/cnn-alzheimers/blob/main/notebooks/18-train-no-TL-weights.ipynb)  

In [ ]:
# mount drive for saving model checkpoints
from google.colab import drive
drive.mount('/content/gdrive')

# load github repo and installing requirements to train the model
!git clone https://github.com/mcqueg/cnn-alzheimers.git
!git config --global user.name "mcqueg"
!git config --global user.email "garrettmccue@gmail.com"
%cd cnn-alzheimers/
!pip install -r requirements.txt

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from datetime import datetime
import os

#import project scripts
from src.models.train_val import train_val

# SET NOTEBOOK PARAMS

# workspaces
COLAB = '/content/cnn-alzheimers/'
GDRIVE = '/content/gdrive/Othercomputers/MacBookPro/cnn-alzheimers/'

# load from colab after mounting repo
TRAIN_DIR = f'{COLAB}data/processed/ADNI/train'
TEST_DIR = f'{COLAB}data/processed/ADNI/test'
LOGS_DIR = f'{COLAB}/logs'

# save to GDRIVE
SAVE_DIR = f'{GDRIVE}models/checkpoints'

# training params
LR = 0.001
VAL_SIZE = 0.5 # splits test dir in half for  validation and testing data
EPOCHS = 500
BATCH = 16
CLASS_NUM = 5

## TensorBoard Training Log

In [ ]:

%load_ext tensorboard
%tensorboard --logdir logs --host 0.0.0.0

## Inception-v3

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

# CREATE DIRS FOR SAVING CHECKPOINTS
NAME = 'inception-v3-no-TL'
now = datetime.now()
start_time = now.strftime("_%d-%m-%Y_%H:%M:%S")
name = f'{NAME}_{start_time}'

os.makedirs(os.path.join(os.path.join(SAVE_DIR,name),'ckpt'), exist_ok=True)
save_weights_path=os.path.join(os.path.join(SAVE_DIR,name), 'ckpt/')
print(f"\nsaving weights at: \n\t{save_weights_path}")

os.makedirs(os.path.join(os.path.join(SAVE_DIR,name), 'config'), exist_ok=True)
config_path=os.path.join(os.path.join(SAVE_DIR,name), 'config')
print(f"\nsaving model architecture at: \n\t{config_path}\n")

# BUILD MODEL
base_model_inception = InceptionV3(input_shape = (256, 256, 3),
                                include_top= False,
                                weights = None)
# choose last layer from the pretrained model to feed into the dense layers
last_layer = base_model_inception.get_layer(index=-1)
last_output = last_layer.output

# add dense layers to be trained for classification by feeding last_output in dense network
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# add a fully connected layer
x = layers.Dense(1024, activation='relu')(x)
# add droput
x = layers.Dropout(0.3)(x)
# add a final layer with softmax for classification
x = layers.Dense(5, activation='softmax')(x)
# create model
inception_model = Model(base_model_inception, x)
# -- save model architecture
model_config = inception_model.to_json()
with open(os.path.join(config_path, f'{name}.json'), 'w') as json_file:
    json_file.write(model_config)

#COMPILE MODEL
inception_model.compile(optimizer = Adam(learning_rate=LR),
                  loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])

#TRAIN MODEL WITH EARLY STOPPING
history_inception = train_val(model=inception_model,
                    name=name,
                    weights_path=save_weights_path,
                    train_dir=TRAIN_DIR,
                    test_dir =TEST_DIR,
                    logs_dir=LOGS_DIR,
                    save_dir=SAVE_DIR,
                    val_size=VAL_SIZE,
                    epochs=EPOCHS,
                    lr=LR,
                    batch_size=BATCH,
                    evaluate=True)



## VGG19

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19

# CREATE DIRS FOR SAVING CHECKPOINTS
NAME = 'vgg19-no-TL'
now = datetime.now()
start_time = now.strftime("_%d-%m-%Y_%H:%M:%S")
name = f'{NAME}_{start_time}'

os.makedirs(os.path.join(os.path.join(SAVE_DIR,name),'ckpt'), exist_ok=True)
save_weights_path=os.path.join(os.path.join(SAVE_DIR,name), 'ckpt/')
print(f"\nsaving weights at: \n\t{save_weights_path}")

os.makedirs(os.path.join(os.path.join(SAVE_DIR,name), 'config'), exist_ok=True)
config_path=os.path.join(os.path.join(SAVE_DIR,name), 'config')
print(f"\nsaving model architecture at: \n\t{config_path}\n")

# BUILD MODEL
base_model_vgg = VGG19(input_shape = (256, 256, 3),
                                include_top= False,
                                weights = None)
# choose last layer from the pretrained model to feed into the dense layers
last_layer = base_model_vgg.get_layer(index=-1)
last_output = last_layer.output

# add dense layers to be trained for classification by feeding last_output in dense network
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# add a fully connected layer
x = layers.Dense(1024, activation='relu')(x)
# add droput
x = layers.Dropout(0.3)(x)
# add a final layer with softmax for classification
x = layers.Dense(5, activation='softmax')(x)
# create model
vgg_model = Model(base_model_vgg, x)
# -- save model architecture
model_config = vgg_model.to_json()
with open(os.path.join(config_path, f'{name}.json'), 'w') as json_file:
    json_file.write(model_config)

#COMPILE MODEL
vgg_model.compile(optimizer = Adam(learning_rate=LR),
                  loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])

#TRAIN MODEL WITH EARLY STOPPING
history_vgg = train_val(model=vgg_model,
                    name=name,
                    weights_path=save_weights_path,
                    train_dir=TRAIN_DIR,
                    test_dir =TEST_DIR,
                    logs_dir=LOGS_DIR,
                    save_dir=SAVE_DIR,
                    val_size=VAL_SIZE,
                    epochs=EPOCHS,
                    lr=LR,
                    batch_size=BATCH,
                    evaluate=True)

## ResNet50

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

# CREATE DIRS FOR SAVING CHECKPOINTS
NAME = 'resnet50-no-TL'
now = datetime.now()
start_time = now.strftime("_%d-%m-%Y_%H:%M:%S")
name = f'{NAME}_{start_time}'

os.makedirs(os.path.join(os.path.join(SAVE_DIR,name),'ckpt'), exist_ok=True)
save_weights_path=os.path.join(os.path.join(SAVE_DIR,name), 'ckpt/')
print(f"\nsaving weights at: \n\t{save_weights_path}")

os.makedirs(os.path.join(os.path.join(SAVE_DIR,name), 'config'), exist_ok=True)
config_path=os.path.join(os.path.join(SAVE_DIR,name), 'config')
print(f"\nsaving model architecture at: \n\t{config_path}\n")

# BUILD MODEL
base_model_resnet = ResNet50(input_shape = (256, 256, 3),
                                include_top= False,
                                weights = None)
# choose last layer from the pretrained model to feed into the dense layers
last_layer = base_model_resnet.get_layer(index=-1)
last_output = last_layer.output

# add dense layers to be trained for classification by feeding last_output in dense network
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# add a fully connected layer
x = layers.Dense(1024, activation='relu')(x)
# add droput
x = layers.Dropout(0.3)(x)
# add a final layer with softmax for classification
x = layers.Dense(5, activation='softmax')(x)
# create model
resnet_model = Model(base_model_resnet, x)
# -- save model architecture
model_config = resnet_model.to_json()
with open(os.path.join(config_path, f'{name}.json'), 'w') as json_file:
    json_file.write(model_config)

#COMPILE MODEL
resnet_model.compile(optimizer = Adam(learning_rate=LR),
                  loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])

#TRAIN MODEL WITH EARLY STOPPING
history_resnet = train_val(model=resnet_model,
                    name=name,
                    weights_path=save_weights_path,
                    train_dir=TRAIN_DIR,
                    test_dir =TEST_DIR,
                    logs_dir=LOGS_DIR,
                    save_dir=SAVE_DIR,
                    val_size=VAL_SIZE,
                    epochs=EPOCHS,
                    lr=LR,
                    batch_size=BATCH,
                    evaluate=True)

## Xception

In [ ]:
from tensorflow.keras.applications.xception import Xception

# CREATE DIRS FOR SAVING CHECKPOINTS
NAME = 'xception-no-TL'
now = datetime.now()
start_time = now.strftime("_%d-%m-%Y_%H:%M:%S")
name = f'{NAME}_{start_time}'

os.makedirs(os.path.join(os.path.join(SAVE_DIR,name),'ckpt'), exist_ok=True)
save_weights_path=os.path.join(os.path.join(SAVE_DIR,name), 'ckpt/')
print(f"\nsaving weights at: \n\t{save_weights_path}")

os.makedirs(os.path.join(os.path.join(SAVE_DIR,name), 'config'), exist_ok=True)
config_path=os.path.join(os.path.join(SAVE_DIR,name), 'config')
print(f"\nsaving model architecture at: \n\t{config_path}\n")

# BUILD MODEL
base_model_xception = Xception(input_shape = (256, 256, 3),
                                include_top= False,
                                weights = None)
# choose last layer from the pretrained model to feed into the dense layers
last_layer = base_model_xception.get_layer(index=-1)
last_output = last_layer.output

# add dense layers to be trained for classification by feeding last_output in dense network
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# add a fully connected layer
x = layers.Dense(1024, activation='relu')(x)
# add droput
x = layers.Dropout(0.3)(x)
# add a final layer with softmax for classification
x = layers.Dense(5, activation='softmax')(x)
# create model
xception_model = Model(base_model_xception, x)
# -- save model architecture
model_config = xception_model.to_json()
with open(os.path.join(config_path, f'{name}.json'), 'w') as json_file:
    json_file.write(model_config)

#COMPILE MODEL
xception_model.compile(optimizer = Adam(learning_rate=LR),
                  loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])

#TRAIN MODEL WITH EARLY STOPPING
history_xception = train_val(model=xception_model,
                    name=name,
                    weights_path=save_weights_path,
                    train_dir=TRAIN_DIR,
                    test_dir =TEST_DIR,
                    logs_dir=LOGS_DIR,
                    save_dir=SAVE_DIR,
                    val_size=VAL_SIZE,
                    epochs=EPOCHS,
                    lr=LR,
                    batch_size=BATCH,
                    evaluate=True)
